# Spotify Extended Streaming History Analysis

## Import Relevant Libraries

In [ ]:
import calendar
from pathlib import Path

import pandas as pd
import plotly.express as px

## Load and Combine Json Files

In [ ]:
folder = "MyData"
paths = Path(folder).glob("*.json")
dfs = [pd.read_json(p) for p in paths]

df_raw = pd.concat(dfs, axis=0)
df_raw.head()

# Define Utilities

In [ ]:
def format_timedelta(td):
    days = td.days
    hours, rem = divmod(td.seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{}d {}h {}m {}s".format(days, hours, minutes, seconds)

## Clean and format data

In [ ]:
# MyData types
df_raw.dtypes

In [ ]:
# Drop redundant columns
df = df_raw.drop([
    'username',
    'platform', 
    'conn_country',
    'ip_addr_decrypted', 
    'user_agent_decrypted',
    'episode_show_name',
    'episode_name',
    'spotify_episode_uri',
    'incognito_mode'], axis=1)

# rename existing columns
df.rename(columns = {
    'master_metadata_track_name':'track_name',
    'master_metadata_album_artist_name':'artist_name',
    'master_metadata_album_album_name':'album_name'
}, inplace = True)

df.head()

In [ ]:
# convert MyData to relevant types
df.ts = pd.to_datetime(df.ts)

df.dtypes

## Earliest Listened Song

In [ ]:
earliest_song = df.iloc[df.ts.argmin()]
earliest_song.to_frame().T.reset_index(drop=True)

## Latest Listened Song

In [ ]:
latest_song = df.iloc[df.ts.argmax()]
latest_song.to_frame().T.reset_index(drop=True)

## Top 10 Listened Songs 

In [ ]:
# Group the MyData by track and artist and calculate the number of plays and total play time
top_songs = df.groupby(['track_name', 'artist_name']).agg({'ms_played': ['count', 'sum']})
top_songs.columns = ['num_plays', 'total_ms_played']

# Convert ms to minutes
top_songs['total_time_pd'] = pd.to_timedelta(top_songs['total_ms_played'], unit='ms')
top_songs['total_time_str'] = top_songs['total_time_pd'].apply(format_timedelta)

# Reset the index to turn the track and artist name into columns
top_songs.reset_index(inplace=True)

# Sort the MyData by number of plays in descending order
# sort by num_plays or ms_played?
top_songs = top_songs.sort_values(by='num_plays', ascending=False)
top_songs.reset_index(inplace=True, drop=True)

top_songs = top_songs.head(10)

fig = px.bar(top_songs,
             y='track_name',
             x='num_plays',
             hover_data=['num_plays', 'artist_name', 'total_time_str'],
             labels={'num_plays': 'Number of Plays', 'track_name': 'Track', 'artist_name': 'Artist', 'total_time_str': 'Total Playtime'},
             orientation='h',
             title='Top 10 Songs',
             template='plotly_dark')

fig.update_layout(yaxis=dict(autorange="reversed"),
                  plot_bgcolor='rgba(0,0,0,0)',
                  paper_bgcolor='rgba(0,0,0,0)')

fig.show()

## Top 10 Listened Artists by Count

In [ ]:
# Group by artist_name and calculate listens count and total playtime
artist_stats = df.groupby('artist_name').agg({'ms_played': 'sum', 'track_name': 'count'}) .rename(columns={'ms_played': 'total_ms_played', 'track_name': 'listen_count'})

# Convert ms to minutes
artist_stats['total_time_pd'] = pd.to_timedelta(artist_stats['total_ms_played'], unit='ms')
artist_stats['total_time_str'] = artist_stats['total_time_pd'].apply(format_timedelta)

# Get top artists by listens count
top_artists = artist_stats.sort_values(by='listen_count', ascending=False).head(10).reset_index()

fig = px.bar(top_artists,
             y='artist_name',
             x='listen_count',
             orientation='h',
             title='Top 10 Artists',
             labels={'listen_count': 'Listens', 'artist_name': 'Artist', 'total_time_str': 'Total Playtime'},
             hover_data=['listen_count', 'total_time_str'],
             template='plotly_dark')

fig.update_layout(yaxis=dict(autorange="reversed"),
                  plot_bgcolor='rgba(0,0,0,0)',
                  paper_bgcolor='rgba(0,0,0,0)')

fig.show()

# Top 10 Listened Songs by Count

In [ ]:
year_min = df.ts.min().year
year_max = df.ts.max().year

for year in range(year_min, year_max + 1):
    top_songs_by_year = df.loc[df.ts.between(f'{year}-01-01', f'{year}-12-31')]
    top_songs_by_year = top_songs_by_year.groupby(['track_name', 'artist_name']).agg(
        {'ms_played': 'sum', 'track_name': 'count'}) \
        .rename(columns={'ms_played': 'total_ms_played', 'track_name': 'listen_count'}).sort_values(by='listen_count',
                                                                                                    ascending=False)
    top_songs_by_year.reset_index(inplace=True)

    # Convert ms to a readable format
    top_songs_by_year['total_time_pd'] = pd.to_timedelta(top_songs_by_year['total_ms_played'], unit='ms')
    top_songs_by_year['total_time_str'] = top_songs_by_year['total_time_pd'].apply(format_timedelta)

    top_songs_by_year = top_songs_by_year.head(10)

    fig = px.bar(top_songs_by_year,
                 y="track_name",
                 x="listen_count",
                 orientation='h',
                 title=f"Top 10 Songs for the Year {year}",
                 labels={'listen_count': 'Listens', 'track_name': 'Song', 'artist_name': 'Artist',
                         'total_time_str': 'Total Playtime'},
                 hover_data=['listen_count', 'artist_name', 'total_time_str'],
                 # include total playtime in the hover data
                 template='plotly_dark')

    fig.update_layout(yaxis=dict(autorange="reversed"),
                      plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor = 'rgba(0,0,0,0)')

    fig.show()

# Top 10 Listened Artists by Year and Count

In [ ]:
for year in range(year_min, year_max + 1):
    # Filter entries for each year
    artists_by_year = df[df['ts'].dt.year == year]
    # Calculate listen count and total_ms_played for each artist
    top_artists_by_year = artists_by_year.groupby('artist_name').agg({'ms_played': 'sum', 'track_name': 'count'})
    top_artists_by_year = top_artists_by_year.rename(
        columns={'ms_played': 'total_ms_played', 'track_name': 'listen_count'})
    top_artists_by_year = top_artists_by_year.sort_values(by='listen_count', ascending=False)

    # Convert total playtime from milliseconds to a readable format
    top_artists_by_year['total_time_pd'] = pd.to_timedelta(top_artists_by_year['total_ms_played'], unit='ms')
    top_artists_by_year['total_time_str'] = top_artists_by_year['total_time_pd'].apply(format_timedelta)

    # Take top 10 artists
    top_artists_by_year = top_artists_by_year.head(10).reset_index()

    # Create a plotly figure 
    fig = px.bar(top_artists_by_year,
                 y="artist_name",
                 x="listen_count",
                 orientation='h',
                 title=f"Top 10 Artists for the Year {year}",
                 labels={'listen_count': 'Listens', 'artist_name': 'Artists', 'total_time_str': 'Total Playtime'},
                 hover_data=['listen_count', 'total_time_str'],
                 # Display the exact listen count and total playtime on hover
                 template='plotly_dark')  # Use the 'plotly_dark' template

    fig.update_layout(yaxis=dict(autorange="reversed"),
                      plot_bgcolor='rgba(0,0,0,0)',
                      paper_bgcolor='rgba(0,0,0,0)')

    fig.show()

# Song Listens Timeline

In [ ]:
df.ts = pd.to_datetime(df.ts)  # assuming ts is in datetime format
df['date'] = df.ts.dt.date

# Count the song plays per date
songs_by_date = df.groupby(df.date)['ts'].size().reset_index(name='listen_count')

# Determine the 'most listened' song for each day
most_listened_song_per_day = df.groupby(['date', 'track_name', 'artist_name']).size().reset_index(name='count')
most_listened_song_per_day = most_listened_song_per_day.sort_values('count', ascending=False).drop_duplicates(['date'])

# Add a new column combining song and artist
most_listened_song_per_day['song_artist'] = most_listened_song_per_day['track_name'] + " - " + \
                                            most_listened_song_per_day['artist_name']

# Merge dataframes
songs_by_date = pd.merge(songs_by_date, most_listened_song_per_day[['date', 'song_artist']], on='date', how='left')

fig = px.line(songs_by_date,
              x='date',
              y='listen_count',
              title=f'Song Listens Timeline ({year_min} - {year_max})',
              hover_data=['song_artist'],
              labels={'date': 'Date', 'listen_count': 'Total Listens', 'song_artist': 'Most Listened Song'},
              template='plotly_dark')

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

fig.show()

# What Years do you Listen to the Most Songs?

In [ ]:
df.ts = pd.to_datetime(df.ts)  # assuming ts is in datetime format
df['year'] = df.ts.dt.year

# Count the song plays per year
songs_by_year = df.groupby(df.year).size().reset_index(name='listen_count')

# Determine the 'most listened' song for each year
most_listened_song_per_year = df.groupby(['year', 'track_name', 'artist_name']).size().reset_index(name='count')
most_listened_song_per_year = most_listened_song_per_year.sort_values('count', ascending=False).drop_duplicates(
    ['year'])

# Add a new column combining song and artist
most_listened_song_per_year['song_artist'] = most_listened_song_per_year['track_name'] + " - " + \
                                             most_listened_song_per_year['artist_name']

# Merge dataframes
songs_by_year = pd.merge(songs_by_year, most_listened_song_per_year[['year', 'song_artist']], on='year', how='left')

fig = px.line(songs_by_year,
              x='year',
              y='listen_count',
              title=f'Total Song Listens by Year  ({year_min} - {year_max})',
              hover_data=['song_artist'],
              labels={'year': 'Year', 'listen_count': 'Listens', 'song_artist': 'Most Listened Song'},
              template='plotly_dark')

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', xaxis=dict(dtick=1))

fig.show()

# What Months do you Listen to the Most Songs?

In [ ]:
df.ts = pd.to_datetime(df.ts)  # assuming ts is in datetime format
df['year'] = df.ts.dt.year
df['month'] = df.ts.dt.month_name()

# Count the song plays per month for each year
songs_by_month = df.groupby(['year', 'month']).size().reset_index(name='listen_count')

# Determine the 'most listened' song for each month of each year
most_listened_song_per_month = df.groupby(['year', 'month', 'track_name', 'artist_name']).size().reset_index(
    name='count')
most_listened_song_per_month = most_listened_song_per_month.sort_values('count', ascending=False).drop_duplicates(
    ['year', 'month'])

# Add a new column combining song and artist
most_listened_song_per_month['song_artist'] = most_listened_song_per_month['track_name'] + " - " + \
                                              most_listened_song_per_month['artist_name']

# Merge dataframes
songs_by_month = pd.merge(songs_by_month, most_listened_song_per_month[['year', 'month', 'song_artist']],
                          on=['year', 'month'], how='left')

# Create a new column 'year_month' for the plot
songs_by_month['year_month'] = songs_by_month['year'].astype(str) + " - " + songs_by_month['month']

pix = px.line(songs_by_month, x='year_month', y='listen_count',
              title=f'Total Song Listens by Month ({year_min} - {year_max})',
              hover_data=['song_artist'], template='plotly_dark',
              labels={'year_month': 'Month', 'listen_count': 'Listens', 'song_artist': 'Most Listened Song'})

pix.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
pix.show()

In [ ]:
df.ts = pd.to_datetime(df.ts)  # assuming ts is in datetime format
df['month'] = df.ts.dt.month

# Count the song plays by month over all years
songs_by_month = df.groupby(df.month).size().reset_index(name='listen_count')

# Determine the 'most listened' song for each month over all years
most_listened_song_per_month = df.groupby(['month', 'track_name', 'artist_name']).size().reset_index(name='count')
most_listened_song_per_month = most_listened_song_per_month.sort_values('count', ascending=False).drop_duplicates(
    ['month'])

# Add a new column combining song and artist
most_listened_song_per_month['song_artist'] = most_listened_song_per_month['track_name'] + " - " + \
                                              most_listened_song_per_month['artist_name']

# Create a dictionary to map month numbers to names
month_dict = {i: month for i, month in enumerate(calendar.month_name[1:], start=1)}

# Replace month numbers with names in both dataframes
songs_by_month['month'] = songs_by_month['month'].replace(month_dict)
most_listened_song_per_month['month'] = most_listened_song_per_month['month'].replace(month_dict)

# Merge the dataframes on the 'month' field
songs_by_month = pd.merge(songs_by_month, most_listened_song_per_month[['month', 'song_artist']], on='month',
                          how='left')

# Generate a plot
fig = px.line(songs_by_month,
              x='month',
              y='listen_count',
              title=f'Total Song Listens by Month (Cumulative)',
              hover_data=['song_artist'],
              labels={'month': 'Month', 'listen_count': 'Listen Count', 'song_artist': 'Top Song'},
              template='plotly_dark')

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', xaxis=dict(dtick=1))

fig.show()

# What Days do you Listen to the Most Songs?

In [ ]:
df.ts = pd.to_datetime(df.ts)  # assuming ts is in datetime format
df['day'] = df.ts.dt.day

# Count the song plays per day over all years
songs_by_day = df.groupby(['day']).size().reset_index(name='listen_count')

# Determine the 'most listened' song for each day over all years
most_listened_song_per_day = df.groupby(['day', 'track_name', 'artist_name']).size().reset_index(name='count')
most_listened_song_per_day = most_listened_song_per_day.sort_values('count', ascending=False).drop_duplicates(['day'])

# Add a new column combining song and artist
most_listened_song_per_day['song_artist'] = most_listened_song_per_day['track_name'] + " - " + \
                                            most_listened_song_per_day['artist_name']

# Merge the dataframes on the 'day' field
songs_by_day = pd.merge(songs_by_day, most_listened_song_per_day[['day', 'song_artist']], on='day', how='left')

# Generate a plot
fig = px.line(songs_by_day,
              x='day',
              y='listen_count',
              title=f'Total Song Listens by Day (Cumulative)',
              hover_data=['song_artist'],
              labels={'day': 'Day of the Month', 'listen_count': 'Listen Count', 'song_artist': 'Top Song'},
              template='plotly_dark')

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', xaxis=dict(dtick=1))

fig.show()

In [ ]:
df_dates = pd.DataFrame()
df_dates['period'] = df['ts'].dt.tz_localize(None).dt.to_period('M').dt.to_timestamp()
df_dates['ms_played'] = df['ms_played']

df_resampled = df_dates.groupby('period')['ms_played'].sum().reset_index()
df_resampled['year'] = df_resampled['period'].dt.year
df_resampled['month'] = df_resampled['period'].dt.strftime('%Y-%m')  # convert 'period' to string

fig = px.line(df_resampled,
              x='month',
              y='ms_played',
              color=df_resampled['year'].astype(str),
              labels={
                  'ms_played': 'Total Play Time (ms)',
                  'month': 'Month',
              },
              title='Listening History Over the Years')

fig.update_layout(template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
df_ = df.set_index('ts')

# Calculate the total number of full listens per day
full_listens_per_day = df_[df_['reason_end'] == 'trackdone']['ms_played'].resample('D').sum() / 30e3

# Calculate the total number of skipped tracks per day
skipped_tracks_per_day = df_[df_['skipped'] == True]['skipped'].resample('D').sum()

# Calculate the average full listens per day and skipped tracks per day
avg_full_listens_per_day = full_listens_per_day.mean()
avg_skipped_tracks_per_day = skipped_tracks_per_day.mean()

# Print the results
print(f'Average full listens per day: {avg_full_listens_per_day}')
print(f'Average skipped tracks per day: {avg_skipped_tracks_per_day}')

In [ ]:
# Calculate the total number of full listens
full_listens = df_[df_['reason_end'] == 'trackdone']['ms_played'].count()

# Calculate the total number of skipped tracks
skipped_tracks = df_[df_['skipped'] == False]['skipped'].count()

# Calculate the ratio of skipped tracks to full listens
skipped_to_full_ratio = skipped_tracks / full_listens

# Print the result
print('Ratio of skipped tracks to full listens:', skipped_to_full_ratio)


In [ ]:
# Sort the DataFrame by timestamp
df_ = df.sort_values('ts')

# Calculate the time difference between each row and the previous row
df_['time_diff'] = df_['ts'].diff()

# Calculate the time difference in seconds
df_['time_diff_sec'] = df_['time_diff'].dt.total_seconds()

# Create a mask for rows where the reason_start is not null
mask = df_['reason_start'].notnull()

# Calculate the time spent listening for each session
df_.loc[mask, 'session_time'] = df_.loc[mask, 'time_diff_sec']

# Forward-fill the session_time column to fill in the null values with the time spent listening in the previous row
df_['session_time'] = df_['session_time'].ffill()

# Group the DataFrame by session and calculate the total time spent listening in each session
session_times = df_.groupby('session_time')['ms_played'].sum() / 1000

# Calculate the average length of listening session
avg_session_length = session_times.mean()

# Print the result
print('Average length of listening session:', avg_session_length, 'seconds')


In [ ]:
# Count the number of songs listened to for each hour of the day
hour_counts = df['ts'].dt.hour.value_counts()

# Find the most popular time of day for listening
most_popular_time = hour_counts.idxmax()

print('The most popular time of day for listening is:', most_popular_time, 'o\'clock')


In [ ]:
df_ = df.set_index('ts')

# Create a new column combining song and artist
df_['track_artist'] = df_['track_name'] + " - " + df_['artist_name']

# Calculate total ms_played for each hour
songs_by_hour = df_.groupby(df_.index.hour)['ms_played'].count()

# Calculate most played track for each hour
most_played_track_per_hour = \
df_.groupby([df_.index.hour, 'track_artist'])['ms_played'].count().groupby(level=0).idxmax().str[1]

# Combine both DFs into one
songs_by_hour_df = pd.DataFrame({
    'ms_played': songs_by_hour.values,
    'Most Played Track': most_played_track_per_hour.values
}, index=songs_by_hour.index)

# Create the bar chart with hover text
fig = px.bar(songs_by_hour_df, x=songs_by_hour_df.index, y='ms_played',
             hover_data=['Most Played Track'],
             labels={'ts': 'Hour of the Day', 'ms_played': 'Listens'}
             )

fig.update_layout(
    title='Most Popular Time of Day for Listening (Cumulative)',
    xaxis_title='Hour of the Day',
    yaxis_title='Number of Songs Listened',
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(dtick=1)
)

ig.show()